<a href="https://colab.research.google.com/github/luiseduardoballarati/MSc-CS-Dissertation/blob/main/svo_function_GH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

This was the notebook used to generate, over a dataset of articles, a dataset containing the svo's of the articles.

# Imports and dataset prep

In [ ]:
!pip install textacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from collections import defaultdict
import re
from datetime import datetime
import spacy
import plotly.express as px
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Bath/Dissertation/The Guardian Datasets/business_guardian_articles_3.csv')

In [ ]:
df.drop(['webUrl'], axis=1, inplace=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Spacy

The series of functions used to extract the svo's.

In [ ]:
# Load SpaCy's English language model
nlp = spacy.load("en_core_web_sm")

def spacy_processor(text):
    # Process the text with SpaCy
    doc = nlp(text)

    # List to hold the results for each sentence
    nested_list = []

    # Iterate over the sentences in the document
    for sent in doc.sents:
        lista = []
        # Extract dependencies for each token in the sentence
        for token in sent:
            lista.extend([(token.text, token.pos_, token.dep_, token.head.text)])

        # Filter out stopwords and punctuation
        filtered_data = [
            (word1, pos, dep, word2) for word1, pos, dep, word2 in lista
            if not (nlp.vocab[word1].is_stop or nlp.vocab[word1].is_punct)
        ]
        # Add the filtered data to the nested list
        nested_list.append(filtered_data)

    #return nested_list
    return nested_list

In [ ]:
# Function to get node attributes
def get_node_attributes(graph, node):
    pos = graph.nodes[node].get('pos', 'N/A')
    dep = graph.nodes[node].get('dep', 'N/A')
    head = graph.nodes[node].get('head', 'N/A')
    return pos, dep, head

def graph_generator(text):
  li = []
  for i in range(len(text)):
  # Create a directed graph
    G = nx.DiGraph()

    # First, create a dictionary to map words to their dependencies
    dep_map = {word: dep for word, pos, dep, head in text[i]}

    # Add edges to the graph and add node attributes for POS, Dependency, Head, and Head Dependency
    for source, pos, dep, target in text[i]:
        head_dep = dep_map.get(target, 'N/A')
        G.add_node(source, pos=pos, dep=dep, head=target, head_dep=head_dep)
        G.add_node(target)  # Ensure the target node is also added
        G.add_edge(source, target, label=dep)

    # Find all weakly connected components in the graph
    components = list(nx.weakly_connected_components(G))

    components_infos = []
    # Print each component with node attributes
    for component in components:
        component_info = []
        for node in component:
            pos, dep, head= get_node_attributes(G, node)
            component_info.append((node, pos, dep, head))
        components_infos.append(component_info)

    li.append(components_infos)
  return li

In [ ]:
def svo_generator(component_info):
    svos = []

    for sentence in component_info:  # Loop over sentences
        for i in range(len(sentence)):  # Loop over components in each sentence
            subjects_adj = []
            verbs_adj = []
            objects_adj = []
            svo = []

            subject = ''
            verb = ''
            objectt = ''

            # check if there is a verb:
            if not any(pos == 'VERB' for word, pos, dep, head in sentence[i]):
              continue

            # first case: nsubj + root + dobj
            for word, pos, dep, head in sentence[i]:
                if (dep == 'nsubj'):
                    subject = word
                if pos == 'VERB' and dep == 'ROOT':
                    verb = word
                if pos == 'NOUN' and dep == 'dobj':
                    objectt = word

            # nsubj but no ROOT
            if verb == '':
              for word, pos, dep, head in sentence[i]:
                  if pos == 'VERB':
                      verb = word

           # nsubj, verb but no dobj
            if objectt == '':
              for word, pos, dep, head in sentence[i]:
                  if pos == 'NOUN' and (dep == 'pobj' or dep == 'iobj'):
                      objectt = word

            for word, pos, dep, head in sentence[i]:
                if (head == subject and pos != 'VERB') or ((pos == 'PROPN' and head != objectt and word != subject and (dep != 'pobj' or dep == 'dobj')) or (pos == 'PRON' and head != objectt and word != subject and (dep != 'pobj' or dep == 'dobj'))):
                    subjects_adj.append(word)
                if head == verb and pos not in ('NOUN', 'PROPN', 'PRON') and head != word:
                    verbs_adj.append(word)
                if head == objectt and pos != 'VERB' or dep == 'dobj':
                      objects_adj.append(word)

            # second case: no subject found:
            if subject == '':
                verb = ''
                objectt = ''
                subjects_adj = []
                verbs_adj = []
                objects_adj = []

                for word, pos, dep, head in sentence[i]:
                  if pos == 'NOUN' and (dep == 'pobj' or dep == 'dobj' or dep == 'iobj') and subject == '':
                    subject = word
                  if pos == 'VERB':
                      verb = word
                  if pos == 'NOUN' and word != subject and head != subject:
                      objectt = word
                #print(f'sub: {subject},v: {verb},o: {objectt}')
                for word, pos, dep, head in sentence[i]:
                  #if head == subject and pos != 'VERB' or ((pos == 'PROPN' or pos == 'PRON') and pos != 'VERB'):
                  if head == subject and pos != 'VERB':
                      subjects_adj.append(word)
                  if head == verb and pos != 'NOUN' and head != word and word not in verbs_adj:
                      verbs_adj.append(word)
                  if (head == objectt) and (pos != 'VERB') and (word not in objects_adj) and (word not in subjects_adj) and (word != subject):
                      objects_adj.append(word)

            #print(f'sub adj: {subjects_adj},v adj: {verbs_adj},o adj: {objects_adj}')

            if subjects_adj:
                merged_subject = " ".join(subjects_adj) + " " + subject
            else:
                merged_subject = subject

            if verbs_adj:
                if verb == 'said':
                  merged_verbs = " ".join(verbs_adj)
                else:
                  merged_verbs = verb + " " + " ".join(verbs_adj)
            else:
                merged_verbs = verb

            if objects_adj:
                merged_objects = " ".join(objects_adj) + " "
            else:
                merged_objects = objectt

            svo = (merged_subject, merged_verbs, merged_objects)
            svos.append(svo)

    return svos

In [ ]:
def svo_creator(content):
  nested_list = spacy_processor(content)
  lista = graph_generator(nested_list)
  svo_list = svo_generator(lista)
  return svo_list

In [ ]:
tqdm.pandas()

def svo_df_creator(df):
  df.dropna(inplace=True)
  df['svo'] = df['content'].progress_apply(lambda x: svo_creator(x))
  df.drop(['type', 'sectionId', 'webTitle', 'content'], axis=1, inplace=True)
  return df

In [ ]:
start_time = time.time()

svo_df_business_guardian_3 = svo_df_creator(df)

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"Time taken to process the sentences: {elapsed_time} seconds")

100%|██████████| 1000/1000 [03:22<00:00,  4.94it/s]

Time taken to process the sentences: 202.34005546569824 seconds


In [ ]:
svo_df_business_guardian_3.to_csv('svo_df_business_guardian_3.csv', index=False)